In [ ]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Load .env from parent directory if needed
env_path = Path(__file__).resolve().parent.parent / ".env"
load_dotenv(dotenv_path=env_path)

# Access variables
OPENAI_API_KEY        = os.getenv("OPENAI_API_KEY")
PINECONE_INDEX_NAME   = os.getenv("PINECONE_INDEX_NAME")
PINECONE_HOST         = os.getenv("PINECONE_HOST")
PINECONE_API_KEY      = os.getenv("PINECONE_API_KEY")
GEMINI_API_KEY        = os.getenv("GEMINI_API_KEY")
K_RETRIEVE            = int(os.getenv("K_RETRIEVE", 5))  # default to 5
OPENAI_API_KEY        = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:


from tqdm import tqdm
from typing import List

import pinecone
import google.generativeai as genai
from sentence_transformers import SentenceTransformer

# ════════════════ INIT MODELS ════════════════
model = SentenceTransformer("intfloat/multilingual-e5-large")  # 1024-D
genai.configure(api_key=GEMINI_API_KEY)

# ════════════════ LOAD CORPUS ════════════════
def split_text_into_chunks(text, chunk_size=900, overlap=100):
    chunks: List = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk.strip())
        start += chunk_size - overlap
    return chunks

# Read the full text
with open("constitucion_nacional.txt", "r", encoding="utf-8") as file:
    text = file.read()

# Split into chunks
chunks = split_text_into_chunks(text)

# Write the chunks to a new file
with open("constitucion_static_chunks.txt", "w", encoding="utf-8") as out_file:
    out_file.write("\n\n".join(chunks))

print(f"Generated {len(chunks)} chunks.")

# ════════════════ EMBEDDING FUNCTION (E5) ════════════════
def embed_texts(texts: List[str]) -> List[List[float]]:
    formatted = [f"passage: {text}" for text in texts]
    return model.encode(formatted, show_progress_bar=True)

print("🔧  Generating embeddings …")
EMBEDS = embed_texts(chunks)


# ════════════════ PINECONE SETUP ════════════════
pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(name=PINECONE_INDEX_NAME, host=PINECONE_HOST)

def upsert_vectors(texts: List[str],
                   vecs: List[List[float]],
                   batch: int = 100):
    for i in tqdm(range(0, len(texts), batch), desc="⬆️  Upserting"):
        batch_vecs = [
            {
                "id": f"id-{j}",
                "values": vecs[j],
                "metadata": {"text": texts[j]}
            }
            for j in range(i, min(i + batch, len(texts)))
        ]
        index.upsert(vectors=batch_vecs)

upsert_vectors(chunks, EMBEDS)


In [ ]:

# ════════════════ RETRIEVE FUNCTION  ════════════════
def retrieve(query: str, k: int = K_RETRIEVE) -> List[str]:
    query_vec = model.encode(f"query: {query}")
    res = index.query(vector=query_vec.tolist(), top_k=k, include_metadata=True)
    return [m.metadata["text"] for m in res.matches]


# ════════════════ GEMINI PRO RAG ════════════════
gemini = genai.GenerativeModel(model_name="gemini-2.0-flash") 

def rag_answer(question: str) -> str:
    context = "\n\n".join(retrieve(question))
    prompt  = f"Contexto:\n{context}\n\nPregunta: {question}\nRespuesta:"
    return gemini.generate_content(prompt).text

In [ ]:
# ════════════════ TEST IT ════════════════
q = "¿Cuáles son las atribuciones del presidente de la Argentina?"
print("\n🔎 Pregunta:", q)
print("\n🧠 Respuesta (Gemini):\n", rag_answer(q))

In [ ]:
import json
import time
from ragas.evaluation import evaluate
from datasets import Dataset
from tqdm import tqdm
import numpy as np



def load_reference_answers(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return {item["question"]: item["answer"] for item in data}



# ════════════════════════════════════════════════
# Construcción del dataset para RAGAS
# ════════════════════════════════════════════════
def prepare_ragas_dataset(reference_answers: dict, tag: str = "") -> Dataset:
    records = []
    for question, reference in tqdm(reference_answers.items(), desc=f"🧪 Procesando {tag}"):
        retrieved = retrieve(question)
        answer = rag_answer(question)
        records.append({
            "user_input": question,
            "retrieved_contexts": retrieved,
            "response": answer,
            "reference": reference
        })
        time.sleep(4)


reference_easy = load_reference_answers("../easy_questions.json")
reference_hard = load_reference_answers("../hard_questions.json")

# Prepare datasets
dataset_easy = prepare_ragas_dataset(reference_easy, tag="fáciles")
dataset_hard = prepare_ragas_dataset(reference_hard, tag="difíciles")

# Save to JSON
dataset_easy.to_json("ragas_easy_2.json", orient="records", lines=False)
dataset_hard.to_json("ragas_hard_2.json", orient="records", lines=False)




In [ ]:
from ragas.embeddings.base import LangchainEmbeddingsWrapper
from sentence_transformers import SentenceTransformer

class CustomE5Embedding(LangchainEmbeddingsWrapper):
    def __init__(self, model_name="intfloat/multilingual-e5-large"):
        self.model = SentenceTransformer(model_name)

    def embed_query(self, texts):
        texts = [f"query: {text}" for text in texts]
        return self.model.encode(texts, convert_to_tensor=False)

    def embed_documents(self, texts):
        texts = [f"passage: {text}" for text in texts]
        return self.model.encode(texts, convert_to_tensor=False)


In [ ]:
from ragas.metrics import (
    LLMContextPrecisionWithoutReference,
    LLMContextPrecisionWithReference,
    NonLLMContextPrecisionWithReference,


    ResponseRelevancy,
    LLMContextRecall,
    Faithfulness
)
from ragas.run_config import RunConfig
from ragas.embeddings.base import embedding_factory
from datasets import load_dataset
import os


# Load datasets
ragas_easy:Dataset = load_dataset("json", data_files="ragas_easy.json", split="train")
ragas_hard:Dataset = load_dataset("json", data_files="ragas_hard.json", split="train")

# Run config
run_config = RunConfig()
custom_embeddings = CustomE5Embedding() 

metrics = [
    LLMContextPrecisionWithReference(),
    LLMContextRecall(),
    Faithfulness()
]

print("\n📊 Evaluando preguntas FÁCILES:")
result_easy = evaluate(
    ragas_easy,
    metrics=metrics,
    run_config=run_config,
    batch_size=1
)
print("✅ Resultados EASY:", result_easy)

print("\n📊 Evaluando preguntas DIFÍCILES:")
result_hard = evaluate(
    ragas_hard,
    metrics=metrics,
    run_config=run_config,
    batch_size=1
)
print("✅ Resultados HARD:", result_hard)



In [ ]:
import numpy as np
print(np.mean(result_easy['llm_context_precision_with_reference']))

In [ ]:
import csv
import numpy as np
from pathlib import Path

csv_path = Path("../results.csv")
write_header = not csv_path.exists()

metric_names = [
    "llm_context_precision_with_reference",
    "context_recall",
    "faithfulness"
]

easy_scores = [np.mean(result_easy[m]) for m in metric_names]
hard_scores = [np.mean(result_hard[m]) for m in metric_names]

with open(csv_path, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    if write_header:
        writer.writerow(["experiment", "dataset"] + metric_names)

    writer.writerow(["experiment_1", "easy"] + easy_scores)
    writer.writerow(["experiment_1", "hard"] + hard_scores)
